In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations


In [ ]:
test = pd.read_csv('test_data_new.csv')
print(1)


In [ ]:
test.info()

In [ ]:


# ====================================================
# Get the difference
# ====================================================
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

# ====================================================
# Read & preprocess data and save it to disk
# ====================================================
def read_preprocess_data_train():
    train = pd.read_csv('train_data_3,5.csv')
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    train_labels = pd.read_csv('train_labels.csv')
    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        print(1)
        train_num_agg[col] = train_num_agg[col].astype(np.float16)
    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        print(1)
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
    # Get the difference
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    del train_num_agg, train_cat_agg, train_diff
    gc.collect()

    train.to_csv('train_lgbt1.csv')

# Read & Preprocess Data
read_preprocess_data_train()

In [ ]:
def get_difference1(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

def read_preprocess_data_test():
    test = pd.read_csv('test_data.csv')
    features = test.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    print('Starting test feature engineer...')
    test_num_agg = test.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    # Transform float64 columns to float32
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        print(1)
        test_num_agg[col] = test_num_agg[col].astype(np.float16)
    # Transform int64 columns to int32
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):

        test_cat_agg[col] = test_cat_agg[col].astype(np.int16)
    # Get the difference
    test_diff = get_difference1(test, num_features)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg, test_diff
    gc.collect()
    # Save files to disk
    test.to_csv('test_lgbt.csv')
read_preprocess_data_test()

In [ ]:
jopa= pd.read_csv('train_lgbt.csv')
jopa.info(memory_usage="deep")

In [ ]:

dtype_dict = {
    'P_2': 'float16',
    'D_39': 'float16',
    'B_1': 'float16',
    'B_2': 'float16',
    'R_1': 'float16',
    'S_3': 'float16',
    'D_41': 'float16',
    'B_3': 'float16',
    'D_42': 'float16',
    'D_43': 'float16',
    'D_44': 'float16',
    'B_4': 'float16',
    'D_45': 'float16',
    'B_5': 'float16',
    'R_2': 'float16',
    'D_46': 'float16',
    'D_47': 'float16',
    'D_48': 'float16',
    'D_49': 'float16',
    'B_6': 'float16',
    'B_7': 'float16',
    'B_8': 'float16',
    'D_50': 'float16',
    'D_51': 'float16',
    'B_9': 'float16',
    'R_3': 'float16',
    'D_52': 'float16',
    'P_3': 'float16',
    'B_10': 'float16',
    'D_53': 'float16',
    'S_5': 'float16',
    'B_11': 'float16',
    'S_6': 'float16',
    'D_54': 'float16',
    'R_4': 'float16',
    'S_7': 'float16',
    'B_12': 'float16',
    'S_8': 'float16',
    'D_55': 'float16',
    'D_56': 'float16',
    'B_13': 'float16',
    'R_5': 'float16',
    'D_58': 'float16',
    'S_9': 'float16',
    'B_14': 'float16',
    'D_59': 'float16',
    'D_60': 'float16',
    'D_61': 'float16',
    'B_15': 'float16',
    'S_11': 'float16',
    'D_62': 'float16',
    'D_63': 'object',
    'D_64': 'object',
    'D_65': 'float16',
    'B_16': 'float16',
    'B_17': 'float16',
    'B_18': 'float16',
    'B_19': 'float16',
    'D_66': 'float16',
    'B_20': 'float16',
    'D_68': 'float16',
    'S_12': 'float16',
    'R_6': 'float16',
    'S_13': 'float16',
    'B_21': 'float16',
    'D_69': 'float16',
    'B_22': 'float16',
    'D_70': 'float16',
    'D_71': 'float16',
    'D_72': 'float16',
    'S_15': 'float16',
    'B_23': 'float16',
    'D_73': 'float16',
    'P_4': 'float16',
    'D_74': 'float16',
    'D_75': 'float16',
    'D_76': 'float16',
    'B_24': 'float16',
    'R_7': 'float16',
    'D_77': 'float16',
    'B_25': 'float16',
    'B_26': 'float16',
    'D_78': 'float16',
    'D_79': 'float16',
    'R_8': 'float16',
    'R_9': 'float16',
    'S_16': 'float16',
    'D_80': 'float16',
    'R_10': 'float16',
    'R_11': 'float16',
    'B_27': 'float16',
    'D_81': 'float16',
    'D_82': 'float16',
    'S_17': 'float16',
    'R_12': 'float16',
    'B_28': 'float16',
    'R_13': 'float16',
    'D_83': 'float16',
    'R_14': 'float16',
    'R_15': 'float16',
    'D_84': 'float16',
    'R_16': 'float16',
    'B_29': 'float16',
    'B_30': 'float16',
    'S_18': 'float16',
    'D_86': 'float16',
    'D_87': 'float16',
    'R_17': 'float16',
    'R_18': 'float16',
    'D_88': 'float16',
    'B_31': 'int64',
    'S_19': 'float16',
    'R_19': 'float16',
    'B_32': 'float16',
    'S_20': 'float16',
    'R_20': 'float16',
    'R_21': 'float16',
    'B_33': 'float16',
    'D_89': 'float16',
    'R_22': 'float16',
    'R_23': 'float16',
    'D_91': 'float16',
    'D_92': 'float16',
    'D_93': 'float16',
    'D_94': 'float16',
    'R_24': 'float16',
    'R_25': 'float16',
    'D_96': 'float16',
    'S_22': 'float16',
    'S_23': 'float16',
    'S_24': 'float16',
    'S_25': 'float16',
    'S_26': 'float16',
    'D_102': 'float16',
    'D_103': 'float16',
    'D_104': 'float16',
    'D_105': 'float16',
    'D_106': 'float16',
    'D_107': 'float16',
    'B_36': 'float16',
    'B_37': 'float16',
    'R_26': 'float16',
    'R_27': 'float16',
    'B_38': 'float16',
    'D_108': 'float16',
    'D_109': 'float16',
    'D_110': 'float16',
    'D_111': 'float16',
    'B_39': 'float16',
    'D_112': 'float16',
    'B_40': 'float16',
    'S_27': 'float16',
    'D_113': 'float16',
    'D_114': 'float16',
    'D_115': 'float16',
    'D_116': 'float16',
    'D_117': 'float16',
    'D_118': 'float16',
    'D_119': 'float16',
    'D_120': 'float16',
    'D_121': 'float16',
    'D_122': 'float16',
    'D_123': 'float16',
    'D_124': 'float16',
    'D_125': 'float16',
    'D_126': 'float16',
    'D_127': 'float16',
    'D_128': 'float16',
    'D_129': 'float16',
    'B_41': 'float16',
    'B_42': 'float16',
    'D_130': 'float16',
    'D_131': 'float16',
    'D_132': 'float16',
    'D_133': 'float16',
    'R_28': 'float16',
    'D_134': 'float16',
    'D_135': 'float16',
    'D_136': 'float16',
    'D_137': 'float16',
    'D_138': 'float16',
    'D_139': 'float16',
    'D_140': 'float16',
    'D_141': 'float16',
    'D_142': 'float16',
    'D_143': 'float16',
    'D_144': 'float16',
    'D_145': 'float16'}
test = pd.read_csv('test_data.csv', dtype = dtype_dict)
test.to_csv('test_7,2gb.csv')

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '/content/data/'
    seed = 42
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe.parquet')
    return train, test

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

# ====================================================
# Train & Evaluate
# ====================================================
def train_and_evaluate(train, test):
    # Label encode categorical features
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    cat_features = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        test[col + '_round2'] = test[col].round(2)
    # Get the difference between last and mean
    num_cols = [col for col in train.columns if 'last' in col]
    num_cols = [col[:-5] for col in num_cols if 'round' not in col]
    for col in num_cols:
        try:
            train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
            test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
        except:
            pass
    # Transform float64 and float32 to float16
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    for col in tqdm(num_cols):
        train[col] = train[col].astype(np.float16)
        test[col] = test[col].astype(np.float16)
    # Get feature list
    features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
    params = {
        'objective': 'binary',
        'metric': CFG.metric,
        'boosting': CFG.boosting_type,
        'seed': CFG.seed,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 40,
    }
    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
        model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 1500,
            verbose_eval = 500,
            feval = lgb_amex_metric
        )
        # Save best model
        joblib.dump(model, f'/content/drive/MyDrive/Amex/Models/lgbm_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
        # Predict validation
        val_pred = model.predict(x_val)
        # Add to out of folds array
        oof_predictions[val_ind] = val_pred
        # Predict the test set
        test_pred = model.predict(test[features])
        test_predictions += test_pred / CFG.n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
    # Compute out of folds metric
    score = amex_metric(train[CFG.target], oof_predictions)
    print(f'Our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
    oof_df.to_csv(f'/content/drive/MyDrive/Amex/OOF/oof_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
    # Create a dataframe to store test prediction
    test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
    test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)

In [ ]:
#seed_everything(42)
#train, test = read_data()
#train_and_evaluate(train, test)